In [2]:
import numpy as np
import pandas as pd
from model import CNN
from pysr import PySRRegressor
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch

/home/croniev/Code/cysec/lib/python3.9/site-packages/juliacall/__init__.py:61: UserWarning: torch was imported before juliacall. This may cause a segfault. To avoid this, import juliacall before importing torch. For updates, see https://github.com/pytorch/pytorch/issues/78829.
  warnings.warn(


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


### Load Model and get Kernels

In [3]:
cnn = CNN()
cnn.load_state_dict(torch.load('cnn.pt'))
print(cnn)

for name, param in cnn.named_parameters():
    if name == 'conv1.weight':
        print(f"amount of kernels of Conv1: {param.shape}")
        kernels1 = param
    if name == 'conv2.weight':
        print(f"amount of kernels of Conv2: {param.shape}")
print(f"kernels of first layer:\n{kernels1}")

/tmp/ipykernel_3964/3040839533.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnn.load_state_dict(torch.load('cnn.pt'))


CNN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (out): Linear(in_features=1568, out_features=10, bias=True)
)
amount of kernels of Conv1: torch.Size([16, 1, 5, 5])
amount of kernels of Conv2: torch.Size([32, 16, 5, 5])
kernels of first layer:
Parameter containing:
tensor([[[[-0.2335, -0.2440, -0.9097, -1.0703, -0.8168],
          [-0.0917,  0.0114, -0.2991, -0.3230,  0.6103],
          [ 0.0865,  0.0287,  0.0781,  0.3111,  0.9103],
          [ 0.0553, -0.7562, -0.8900, -0.9722, -0.9644],
          [ 0.0107, -0.6425, -0.8105, -0.5706, -1.1133]]],


        [[[-0.0280, -0.6446, -1.0783, -0.0510, -0.1909],
          [-1.2921, -1.5182, -0.0516,  0.5699, -0.5167],
          [-

### Load Dataset and get results

In [4]:
test_data = datasets.MNIST(root='data', train=False, transform=ToTensor(),)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=10, shuffle=True, num_workers=1)
samples, labels = next(iter(test_loader))

cnn.eval()
with torch.no_grad():
    results = cnn(samples)

### Prepare Data for PySR
Extract the 5x5 submatrices (incl. padding) from images to use them as input

In [38]:
# Take every image and split it into 5x5 submatrices => np.array.shape = (7840, 25)
# 25 <- flattened 5x5 patch
# 7840 <- 28 * 28 patches per image * 10 images (batch_size)
kernel_size = 5
X = None
for x in samples:
    x = torch.nn.functional.pad(input=x[0], pad=(2, 2, 2, 2), mode="constant", value=0)
    for i, j in np.ndindex((x.size()[0] - kernel_size + 1, x.size()[1] - kernel_size + 1)):
        slice = x[i:i + kernel_size, j:j + kernel_size]
        if X is None:
            # X = np.array([slice.numpy().flatten()])
            X = np.array([slice.numpy()])
        else:
            # X = np.concatenate((X, [slice.numpy().flatten()]))
            X = np.concatenate((X, [slice.numpy()]))

print(X.shape)

# Get the result for every 5x5 submatrix for each kernel => np.array.shape = (16, 7840)
# 16 <- amount of kernels in the first layer
y = results['relu1'].numpy().transpose(1, 0, 2, 3).reshape(16, 7840)
print(y.shape)

(7840, 5, 5)
(16, 7840)


#### Physical Features of submatrices
Multiply Matrix with position/speed matrix so that positional data is encoded.

In [66]:
d = np.fromfunction(lambda i, j: np.sqrt(i**2 + j**2), (5, 5))  # Matrix with distances to top-left corner
print(d)
# Xpos = np.vectorize(lambda m: m * d.flatten())(X)
Xpos = np.array([m * d for m in X])
print(X[300])
print(Xpos[300])

[[0.         1.         2.         3.         4.        ]
 [1.         1.41421356 2.23606798 3.16227766 4.12310563]
 [2.         2.23606798 2.82842712 3.60555128 4.47213595]
 [3.         3.16227766 3.60555128 4.24264069 5.        ]
 [4.         4.12310563 4.47213595 5.         5.65685425]]
[[0.70980394 0.7372549  0.85882354 0.99607843 0.45490196]
 [0.00392157 0.00784314 0.8980392  0.8980392  0.14117648]
 [0.         0.         0.9019608  0.6        0.        ]
 [0.         0.47843137 0.827451   0.7411765  0.6156863 ]
 [0.23921569 0.9843137  0.99607843 0.99607843 0.99607843]]
[[0.00000000e+00 7.37254918e-01 1.71764708e+00 2.98823529e+00
  1.81960785e+00]
 [3.92156886e-03 1.10918717e-02 2.00807675e+00 2.83984937e+00
  5.82085527e-01]
 [0.00000000e+00 0.00000000e+00 2.55113036e+00 2.16333085e+00
  0.00000000e+00]
 [0.00000000e+00 1.51293285e+00 2.98341697e+00 3.14454552e+00
  3.07843149e+00]
 [9.56862748e-01 4.05842946e+00 4.45459817e+00 4.98039216e+00
  5.63467051e+00]]


#### Sum up rows and cols  ($f(A) \rightarrow x, A \in \mathbb{R}^{5 \times 5}, x \in \mathbb{R}^{10}$)

In [69]:
def sums_rows_and_cols(m):
    return np.concatenate((m.sum(axis=1), m.sum(axis=0)))

Xrc = np.array([sums_rows_and_cols(m) for m in X])
print(Xrc.shape)
Xposrc = np.array([sums_rows_and_cols(m) for m in Xpos])
print(Xposrc.shape)
print(Xrc[300])

(7840, 10)
(7840, 10)
[3.7568629 1.9490197 1.5019608 2.662745  4.211765  0.9529412 2.2078433
 4.4823527 4.231373  2.2078433]


### Symbolic Regression
#### Over all 16 Kernels

In [70]:
regr_functions = pd.DataFrame()
regr_functions.index.names = ['complexity']
for i in range(16):
    regr = PySRRegressor(
        niterations=40,
        binary_operators=["+", "*", "-", "/"],
        unary_operators=[
            "cos",
            "exp",
            "sin",
            "square",
            "cube",
            "inv(x) = 1/x",  # Julia syntax
        ],
        extra_sympy_mappings={"inv": lambda x: 1 / x},  # Sympy syntax
        elementwise_loss="loss(prediction, target) = (prediction - target)^2",  # Julia syntax
        warm_start=False,
        verbosity=0,
        temp_equation_file=True,
    )

    # regr.fit(X.reshape((X.shape[0], 25)), y[i])  # Input data as is
    # regr.fit(Xpos.reshape((Xpos.shape[0], 25)), y[i])  # Input data coded for postition
    regr.fit(Xposrc, y[i])  # Input data coded for position and summed
    # print(regr.equations_)
    regr_functions.insert(loc=i, column=f'Kernel {i}', value=regr.equations_['equation'])
    print(f"Done with Kernel {i} | {i+1/16 * 100}%")

print(regr_functions)

/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2059: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, and consider using a larger `maxsize`.
  warnings.warn(


Done with Kernel 0 | 0.0%


/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2059: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, and consider using a larger `maxsize`.
  warnings.warn(


Done with Kernel 1 | 0.0625%


/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2059: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, and consider using a larger `maxsize`.
  warnings.warn(


...

Done with Kernel 13 | 0.8125%


/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2059: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, and consider using a larger `maxsize`.
  warnings.warn(


Done with Kernel 14 | 0.875%


/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2059: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, and consider using a larger `maxsize`.
  warnings.warn(


Done with Kernel 15 | 0.9375%
                                                     Kernel 0  \
complexity                                                      
0                                                 -0.75026596   
1                                          square(-0.0374831)   
2                                          0.00068825914 * x2   
3                                   square(-0.027123682) * x2   
4                                   -0.0014752903 * (x6 - x2)   
5                     -0.0014752903 * ((x3 / 1.2241117) - x2)   
6                    x2 * square(inv(27.069881 + square(x3)))   
7           (cube(0.017324438) / exp(x0)) * square(square(...   
8           (square(-0.1099104) / ((x5 - -0.6081664) + x8)...   
9           sin(sin(square(0.010120501 / (exp(x7) + x0)) *...   
10          sin(sin(sin((0.010120501 / (x0 + (exp(x3) / 1....   
11          sin((square(0.010120501 / (exp(x7) + square(0....   
12          sin(sin(square(0.010120501 / ((exp(x7) / squar..

In [71]:
# regr_functions.to_csv('regression_conv1_relu1.csv')
regr_functions.to_csv('regression_conv1_relu1_posrc.csv')

#### Over the first kernel multiple times (Stability check)

In [ ]:
regr_stability = pd.DataFrame()
regr_stability.index.names = ['complexity']
for i in range(10):
    regr = PySRRegressor(
        niterations=40,
        binary_operators=["+", "*", "-", "/"],
        unary_operators=[
            "cos",
            "exp",
            "sin",
            "square",
            "cube",
            "inv(x) = 1/x",  # Julia syntax
        ],
        extra_sympy_mappings={"inv": lambda x: 1 / x},  # Sympy syntax
        elementwise_loss="loss(prediction, target) = (prediction - target)^2",  # Julia syntax
        warm_start=False,
        verbosity=0,
        temp_equation_file=True,
    )

    regr.fit(X, y[0])
    # print(regr.equations_)
    regr_stability.insert(loc=i, column=f'Iteration {i}', value=regr.equations_['equation'])
    print(regr_stability)

In [ ]:
regr_stability.to_csv('stability_conv1_relu1_kernel1.csv')